In [ ]:
# Performance is dependent on choice of model and effective prompt generation.
# Open-Source Models are found from Huggingface.

In [62]:
## CONFIG
DEFAULT_MODEL="google/flan-t5-small"
MODEL1="google/flan-t5-small"
MODEL2="google/flan-t5-large"

#Open Source Examples
#flan-t5-small, flan-t5-base, flan-t5-large, flan-t5-xl, flan-t5-xxl
#llama-2-7b, llama-2-13b, llama-2-70b
#bloom-560m, bloom-1b1, bloom-7b1, bloomz-7b1-mt

#Closed Source Examples
#GPT-4


PROMPT="I need a bracket that can support 20kg with two mounting holes."


DEFAULT_PROMPT_TEMPLATE=("Extract design parameters as a Python dictionary with string keys and values.\n\n"
"User's input:\n"
"{user_input}\n\n")

PROMPT_TEMPLATE1=("You are an AI assistant that extracts design parameters from text.\n\n"
    "Instructions: The user will describe a design they need, mentioning various requirements. "
    "Please read the user's text and identify any key design parameters, requirements, or features. "
    "Extract key design parameters from the following input and return them as a valid Python dictionary with string keys and string or numeric values only. "
    "Do not include any explanation, just the dictionary. "
    "If some parameters are given in natural language, just return them as strings. "
    "If quantitative values are mentioned, return them as strings with units. "
    "Be as general as possible; do not assume categories in advance.\n\n"
    "User's input:\n"
    "{user_input}\n\n"
    "Output:")

PROMPT_TEMPLATE2 = (
    "You are an AI assistant that extracts design parameters from text.\n\n"
    "Instructions: The user will describe a design they need, mentioning various requirements.\n\n"
    "Please read the user's text and identify any key design parameters, requirements, or features.\n\n"
    "Extract key design parameters from the following input and return them as a valid Python dictionary with string keys and string or numeric values only.\n\n"
    "Do not include any explanation, just the dictionary.\n\n"
    "If some parameters are given in natural language, just return them as strings.\n\n"
    "If quantitative values are mentioned, return them as strings with units.\n\n"
    "Be as general as possible; do not assume categories in advance.\n\n"
    "Follow the examples below:\n\n"
    
    "Example 1:\n"
    "Input: I want a shelf that can hold 15kg and has three levels.\n"
    "Output: {{'type': 'shelf', 'capacity': '15kg', 'levels': '3'}}\n\n"
    
    "Example 2:\n"
    "Input: A pipe clamp that supports 500N of force and includes a hinge.\n"
    "Output: {{'type': 'pipe clamp', 'capacity': '500N', 'features': 'includes a hinge'}}\n\n"
    
    "Example 3:\n"
    "Input: I need a frame to hold a 10kg painting with a gold finish.\n"
    "Output: {{'type': 'frame', 'capacity': '10kg', 'features': 'gold finish'}}\n\n"
    
    "Example 4:\n"
    "Input: I want a mount for a TV that weighs 30kg with adjustable tilt.\n"
    "Output: {{'type': 'TV mount', 'capacity': '30kg', 'features': 'adjustable tilt'}}\n\n"
    
    "Now, process the following input and extract the design parameters:\n\n"
    "Input: {user_input}\n"
    "Output:"
)



In [33]:
# testing transformers is working
from transformers import pipeline
pipe = pipeline("sentiment-analysis")
print(pipe("This is working perfectly!"))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9998660087585449}]


In [ ]:
# Open Source Template

In [53]:
import ast
# Load a suitable instruction-following model (replace with a model you have access to)
# For example, using a LLaMA-2 or a Flan-T5 large model:
# Note: This is a placeholder. You need an appropriate model & weights that support instruction-following.
# In practice, you might load something like:
# pipe = pipeline("text-generation", model="google/flan-t5-xxl", device_map="auto")
#
# Or if using an OpenAI API (not shown here since it's a different library),
# you'd call the openai.Completion.create with a prompt.
#
# For demonstration, let's assume `pipe` is a pipeline capable of following instructions.

def extract_design_parameters(user_input: str, prompt_template: str = DEFAULT_PROMPT_TEMPLATE):
    """
    Extracts design parameters from the user's input using a specified prompt template.

    Args:
        user_input (str): The user's input text describing the design requirements.
        prompt_template (str): The template for the prompt, formatted with the user_input.

    Returns:
        dict: A dictionary containing both the raw response and the stripped response, along with parsed parameters if possible.
    """
    # Format the prompt with the user's input
    prompt = prompt_template.format(user_input=user_input)
    
    # Generate the response
    response = pipe(prompt)[0]['generated_text']
    stripped_response = response.strip()  # Remove any leading/trailing whitespace

    # Initialize the result dictionary
    result = {
        "raw_response": response,
        "stripped_response": stripped_response,
        "parsed_parameters": None  # Default to None until parsing is attempted
    }

    # Try to parse the stripped response as a dictionary
    try:
        extracted_params = ast.literal_eval(stripped_response)
        if isinstance(extracted_params, dict):
            result["parsed_parameters"] = extracted_params
        else:
            # If parsing doesn't result in a dict, store the stripped response
            result["parsed_parameters"] = {"parameters": stripped_response}
    except:
        # If parsing fails, store the stripped response as is
        result["parsed_parameters"] = {"parameters": stripped_response}

    return result


In [65]:
pipe = pipeline("text2text-generation", model=MODEL1, max_new_tokens=200)

Device set to use mps:0


In [66]:
user_input = PROMPT
extracted = extract_design_parameters(user_input, DEFAULT_PROMPT_TEMPLATE)
print(extracted['raw_response'])
print(extracted['stripped_response'])
print(extracted['parsed_parameters'])

VB
VB
{'parameters': 'VB'}


In [67]:
user_input = PROMPT
extracted = extract_design_parameters(user_input, PROMPT_TEMPLATE1)
print(extracted['raw_response'])
print(extracted['stripped_response'])
print(extracted['parsed_parameters'])

'1'
'1'
{'parameters': "'1'"}


In [68]:
user_input = PROMPT
extracted = extract_design_parameters(user_input, PROMPT_TEMPLATE2)
print(extracted['raw_response'])
print(extracted['stripped_response'])
print(extracted['parsed_parameters'])

'type': bracket, 'capacity': '20kg', 'capacity': '2', 'mounting': '2'
'type': bracket, 'capacity': '20kg', 'capacity': '2', 'mounting': '2'
{'parameters': "'type': bracket, 'capacity': '20kg', 'capacity': '2', 'mounting': '2'"}


In [ ]:
# performance depends on model

In [69]:
pipe = pipeline("text2text-generation", model=MODEL2, max_new_tokens=400)

Device set to use mps:0


In [70]:
user_input = PROMPT
extracted = extract_design_parameters(user_input, PROMPT_TEMPLATE2)
print(extracted['raw_response'])
print(extracted['stripped_response'])
print(extracted['parsed_parameters'])

'bracket', 'capacity', '20kg', 'two mounting holes'
'bracket', 'capacity', '20kg', 'two mounting holes'
{'parameters': "'bracket', 'capacity', '20kg', 'two mounting holes'"}


In [ ]:
# Closed Source Code

In [93]:
#!pip install --upgrade openai

In [ ]:
# Run the below code in your terminal to protect your key.
# export OPENAI_API_KEY="your-api-key"


In [82]:
import os
api_key = os.getenv("OPENAI_API_KEY")

In [91]:
openai.api_key=api_key

In [92]:
import openai

def extract_design_parameters(user_input: str):
    """
    Extracts design parameters from the user input using GPT-3.5 or GPT-4.
    """
    prompt = (
        "You are an AI assistant that extracts design parameters from text.\n\n"
        "Instructions: The user will describe a design they need, mentioning various requirements.\n\n"
        "Extract the design parameters and return them as a Python dictionary with string keys and values only.\n\n"
        f"Input: {user_input}\n\n"
        "Output:"
    )
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Or "gpt-4" for better quality
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    return response['choices'][0]['message']['content'].strip()

# Example usage
user_input = "I need a bracket that can support 20kg with two mounting holes."
print(extract_design_parameters(user_input))


In [90]:
user_input = "I need a bracket that can support 20kg with two mounting holes."
print(extract_design_parameters(user_input))

In [95]:
# # I am pasting the response that I got from web api using GPT-4-o.
# prompt="You are an AI assistant that extracts design parameters from text."
#         "Instructions: The user will describe a design they need, mentioning various requirements."
#         "Extract the design parameters and return them as a Python dictionary with string keys and values only."

# user_input = "I need a bracket that can support 20kg with two mounting holes."
# output=
# {
#     "type": "bracket",
#     "load_capacity": "20kg",
#     "mounting_holes": "2"
# }
# #which is good.
